<a href="https://colab.research.google.com/github/thiagodma/Cluster-Normas/blob/deep-clustering/get_sentence_representation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai import *
from fastai.text import *

In [0]:
path = untar_data(URLs.IMDB_SAMPLE)
data_lm = TextLMDataBunch.from_csv(path, 'texts.csv')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [0]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.324240,3.882482,0.289078,00:11


In [0]:
xb,yb = learn.data.one_item('i did not like this movie because it was really bad.')

In [0]:
sentence = torch.cuda.LongTensor(xb.tolist()[0]).unsqueeze(0)

In [0]:
raw_outputs, outputs = learn.model[0].forward(sentence)

In [0]:
sentence_rep = outputs[0][-1][-1]
sentence_rep.shape

torch.Size([1152])